In [2]:
import json
import requests
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from api_keys import weather_api_key

COVID tracking project API
https://covidtracking.com/data/download

In [10]:
#url = "https://api.covidtracking.com/v2/states/ak/daily.json"

#response = requests.get(url)
#response_json = response.json()
#pprint(requests.get(url).json())



In [177]:
covid_data = pd.read_csv("Folder1/all-states-history.csv")
covid_data.head()

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,3/7/21,AK,305.0,NaN,0,NaN,1293.0,1293.0,33.0,0,...,1731628.0,0,NaN,NaN,NaN,NaN,NaN,0,1731628.0,0
1,3/7/21,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,NaN,NaN,119757.0,NaN,2323788.0,2347,NaN,0
2,3/7/21,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,NaN,NaN,NaN,481311.0,NaN,0,2736442.0,3380
3,3/7/21,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,3/7/21,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,NaN,444089.0,NaN,3842945.0,14856,7908105.0,45110


In [189]:
removed_columns_df = covid_data[["date", "state", "death"]]
removed_columns_df.head()

,date,state,death
0,3/7/21,AK,305.0
1,3/7/21,AL,10148.0
2,3/7/21,AR,5319.0
3,3/7/21,AS,0.0
4,3/7/21,AZ,16328.0


In [182]:
removed_columns_df.count()

date     20780
state    20780
death    19910
dtype: int64

In [186]:
clean_death_data_df = removed_columns_df.dropna(how='any')

In [187]:
clean_death_data_df.count()

date     19910
state    19910
death    19910
dtype: int64

In [193]:
clean_death_data_df

,date,state,death
0,3/7/21,AK,305.0
1,3/7/21,AL,10148.0
2,3/7/21,AR,5319.0
3,3/7/21,AS,0.0
4,3/7/21,AZ,16328.0
...,...,...,...
20684,2/14/20,NJ,0.0
20689,2/13/20,NJ,0.0
20694,2/12/20,NJ,0.0
20699,2/11/20,NJ,0.0


In [205]:
unique_state = clean_death_data_df['state'].unique()
unique_state

array(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
       'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA',
       'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE',
       'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI',
       'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV',
       'WY'], dtype=object)

In [232]:
as_data = clean_death_data_df[clean_death_data_df.state != 'AS']
dc_data = as_data[as_data.state != 'DC']
gu_data = dc_data[dc_data.state != 'GU']
mp_data = gu_data[gu_data.state != 'MP']
pr_data = mp_data[mp_data.state != 'PR']
clean_data = pr_data[pr_data.state != 'VI']

In [233]:
unique = clean_data['state'].unique()
unique

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI',
       'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV',
       'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [234]:
clean_data

,date,state,death
0,3/7/21,AK,305.0
1,3/7/21,AL,10148.0
2,3/7/21,AR,5319.0
4,3/7/21,AZ,16328.0
5,3/7/21,CA,54124.0
...,...,...,...
20684,2/14/20,NJ,0.0
20689,2/13/20,NJ,0.0
20694,2/12/20,NJ,0.0
20699,2/11/20,NJ,0.0


In [239]:
clean_data = clean_data.sort_values("date", ascending=False)

In [243]:
clean_deathperday_data = clean_data.reset_index(drop=True)

In [244]:
clean_deathperday_data

,date,state,death
0,9/9/20,AK,42.0
1,9/9/20,SC,2942.0
2,9/9/20,NC,2958.0
3,9/9/20,ND,160.0
4,9/9/20,NH,433.0
...,...,...,...
17821,1/1/21,MD,5942.0
17822,1/1/21,MI,13018.0
17823,1/1/21,MN,5323.0
17824,1/1/21,MO,5540.0


In [245]:
clean_deathperday_data.to_csv("Folder2/cleandeathperday.csv", index=False, header=True)

COVID Tracking project - Race CSV

In [ ]:
race_data = pd.read_csv("Folder1/CRDT Data-CRDT.csv")
covid_data.head()